In [1]:
import pandas as pd

In [19]:
df_all = pd.read_csv('LSDS-8_Comprehensive_Metabolic_Panel_CMP_TRANSFORMED_all_astronauts.csv')

In [16]:
df_all.shape

(28, 60)

In [7]:
# check for missing values

missing = df_all.isnull().sum()
missing = missing[missing > 0]
missing

total_bilirubin_value_milligram_per_deciliter                                        1
bun_to_creatinine_ratio_value                                                       22
bun_to_creatinine_ratio_range_min                                                    1
bun_to_creatinine_ratio_range_max                                                    1
egfr_african_american_range_max_milliliter_per_minute_per_1.73_meter_squared        28
egfr_non_african_american_range_max_milliliter_per_minute_per_1.73_meter_squared    28
dtype: int64

In [13]:
# fill missing values with median
# bilirubin values can be skewed, so median is more robust than mean

df_all["total_bilirubin_value_milligram_per_deciliter"] = (
    df_all["total_bilirubin_value_milligram_per_deciliter"].fillna(
        df_all["total_bilirubin_value_milligram_per_deciliter"].median()
    )
)

In [ ]:
# do we drop this ? 22/28 are missing
df_all["bun_to_creatinine_ratio_value"] = (
    df_all["bun_to_creatinine_ratio_value"]
    .fillna(df_all["bun_to_creatinine_ratio_value"].median())
)

df_all["bun_to_creatinine_ratio_range_min"] = (
    df_all["bun_to_creatinine_ratio_range_min"]
    .fillna(df_all["bun_to_creatinine_ratio_range_min"].median())
)

df_all["bun_to_creatinine_ratio_range_max"] = (
    df_all["bun_to_creatinine_ratio_range_max"]
    .fillna(df_all["bun_to_creatinine_ratio_range_max"].median())
)

In [17]:
# drop empty columns 

# ---- Drop columns ----
df_all = df_all.drop(
    columns=[
        "egfr_african_american_range_max_milliliter_per_minute_per_1.73_meter_squared",
        "egfr_non_african_american_range_max_milliliter_per_minute_per_1.73_meter_squared",
    ],
    errors="ignore",  # just in case column missing
)

In [18]:
# check for missing values

missing = df_all.isnull().sum()
missing = missing[missing > 0]
missing

Series([], dtype: int64)

In [20]:
df_all.to_csv('/Users/mrinalgoel/Desktop/sourcing_data/Inspiration-Health-Data/cleaned_data/LSDS-8_Comprehensive_Metabolic_Panel_CMP_TRANSFORMED_all_astronauts_filled.csv', index=False)   